# Crashlytics

In [18]:
try:
    import pyspark
except ModuleNotFoundError:
    !pip3 install pyspark
    import pyspark
try:
    import pandas as pd
except ModuleNotFoundError:
    !pip3 install pandas
    import pandas as pd
    import csv

In [10]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, LongType, IntegerType, FloatType
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [14]:
#May need to install java for this to work
ss=SparkSession.builder.master("local").appName("crashlytics").getOrCreate()

In [24]:
raw_df = ss.read.csv("smallest_crash_data.csv", header=True, inferSchema=True)

In [40]:
#raw_df.printSchema()
#raw_df.show(5)

## Imbalance in Data

Crash severity of 2 or 3 is much more common than any other severity ranking, and crash durations (duration of impact on traffic) are recorded as 30 minutes the majority of the time. This shows that the data is imbalanced, which we will have to consider when building our classifier.

In [39]:
severity_count = raw_df.groupBy(F.col("Severity")).count()
severity_count.show()

time_count = raw_df.withColumn("time_diff", (F.col("Start_Time").cast("long") - F.col("End_Time").cast("long"))/60).groupBy(F.col("time_diff")).count()
time_count.orderBy(F.col("count"), ascending=False).show()

+--------+-----+
|Severity|count|
+--------+-----+
|       1|    1|
|       3|   81|
|       2|  118|
+--------+-----+

+------------------+-----+
|         time_diff|count|
+------------------+-----+
|             -30.0|  151|
|             -45.0|   21|
|             -60.0|    2|
|            -235.0|    1|
|            -826.0|    1|
|-74.61666666666666|    1|
|-64.53333333333333|    1|
|            -314.0|    1|
|            -137.0|    1|
|            -198.0|    1|
|-72.98333333333333|    1|
|             -92.0|    1|
|            -350.0|    1|
|            -177.0|    1|
|            -109.0|    1|
|            -227.0|    1|
|             -72.0|    1|
|            -871.0|    1|
|            -186.0|    1|
|            -156.0|    1|
+------------------+-----+
only showing top 20 rows

